# Лабораторная работа по Теории кодирования
## Лабораторная работа №4
Выполнили студенты группы 6401-010302D

Баловнева Юлия

Богатырев Дмитрий


In [157]:
import numpy as np
import itertools


def mult(m1, m2):
    '''Умножает две матрицы xor-ом и возвращает результат
    :param:
        m1: первая матрица
        m2: вторая матрица
        
        :return: C: возвращает матрицу - xor-произведение двух матриц 
    '''
    A = np.array(m1, dtype=int)
    B = np.array(m2, dtype=int)
    C = (A@B)%2
    return np.array(C, dtype=bool)

def code_length(matr):
    '''Вычисляет кодовое расстояние массива переданных слов
    :param:
        matr: матрица - двумерный bool-массив - масиив двоичных слов
        
        :return: d: кодовое расстояние. 
    '''
    d = len(matr[0])
    for i in range(len(matr)-1):
        for j in range(i+1,len(matr)):
            # Ищем минимальное число единиц в множестве произведений i и j строк
            d = min(d,sum(np.bitwise_xor(matr[i],matr[j])))
    return d

def create_verif_matr(X):
    '''
    Создает проверочную матрицу, присоединяя к матрице X единичную матрицу сверху 
    :param X: матрица X
    :return: проверочную матрицу
    '''
    k = X.shape[1]
    P = np.eye(k,dtype=bool)
    return np.vstack([P,X])

def create_rref_matr(X):
    '''
    Создает порождающую матрицу, присоединяя к матрице X единичную матрицу слева
    :param X: матрица X
    :return: порождающую матрицу
    '''
    k = X.shape[0]
    P = np.eye(k,dtype=bool)
    return np.hstack([P,X])

In [158]:
class LinearCode:
    ''' Класс линейных кодов.

    Атрибуты:\n
    matr: копия исходной матрицы\n
    k: количество строк матрицы\n
    n: количество столбцов матрицы
    
    '''
    # Конструктор
    def __init__(self, n_array):
        self.matr = np.array(n_array, dtype=bool)
        self.n = n_array.shape[1]
        self.k = n_array.shape[0]
    
    def find_first(self, line):
        '''Поиск первого ненулевого элемента в строке.
        
        :param:
        line: строка матрицы.
        
        :return: h: возвращаем позицию первого ненулевого элемента в строке. 
        '''
        # Задаем h = количеству элементов в сторке
        h = self.n
        # В строке line перебираем элементы пока не натолкнемся на 1
        for i in range(self.n):
            if self.matr[line,i]:
                h = i
                break
        # Возвращаем позицию найденной единицы в строке
        return h
    
    def xor_swap(self, line1, line2):
        '''Меняет две строки местами с помощью поэлементного XOR.

        :param:
            line1: первая строка.\n
            line2: вторая строка.

        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line1,w] ^= self.matr[line2,w]
            self.matr[line2,w] ^= self.matr[line1,w]
            self.matr[line1,w] ^= self.matr[line2,w]


    def sum(self, line1, line2):
        '''Складывает первую переданную строку со второй.

        :param:
            line1: первая строка.\n
            line2: вторая строка.
        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line2,w] ^= self.matr[line1,w]

    def __str__(self):
        '''Собирает строку (смена типа от bool к string).

        :return: s: строка с матрицей.            
        '''
        s = ''
        for i in range(self.k):
            s += '['
            for j in range(self.n):
                if self.matr[i,j]: s += "1 "
                else: s += "0 "
            s += "\b]\n"
        return s
    
    def fillrand(self):
        '''Заполняет матрицу случайным образом нулями и единицами.
        
        '''
        rand = np.random
        for i in range(self.k):
            for j in range(self.n):
                self.matr[i,j] = np.round(rand.random())

    def sort(self):
        '''Сортировка строк матрицы.
        '''
        # sort
        for i in range(self.k):
            for j in range(self.k-i-1):
                if self.find_first(j) > self.find_first(j+1):
                    self.xor_swap(j, j+1)

    def ref(self):
        '''Приводит матрицу к ступенчатому виду.
        
            ---
            Ступенчатая матрица - такая матрица, что
            * все ненулевые строки располагаются над всеми чисто нулевыми строками
            * ведущий элемент (первый, считая слева направо, ненулевой элемент строки)
            каждой ненулевой строки располагается строго правее ведущего элемента в
            строке, расположенной выше данной.
            ---
        '''
        # Проходит по всем строкм матрицы
        for i in range(self.k-1):
            # Сравнивает рабочую строку с текущей
            for j in range(i+1, self.k):
                # Запоминаем первые элементы
                first_i = self.find_first(i)
                first_j = self.find_first(j)
                # Если они равны, то делаем xor со строкой
                # Поскольку рабочая строка всегда ниже текущей, то при xor первый элемент всех нижних строк зануляется
                if first_i == first_j:
                    self.sum(i,j)
                # Если встретили строку, у которой первый элемент раньше чем у текущей, то меняем строки местами
                elif first_i > first_j:
                    self.xor_swap(i,j)
                # Иначе все хорошо, идем дальше
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
        

    def rref(self):
        '''Приводит матрицу к приведенному ступенчатому виду. 
        
            ---
            Ступенчатая матрица называется приведенной, если матрица,  не имеет нулевых строк, 
            и все ведущие элементы ее строк равны единице. При этом все элементы основных столбцов, 
            помимо ведущих элементов, являются нулями.
            
        '''
        # Сначала приводим матрицу к ступенчатому виду
        self.ref()
        # Дальше для всех строк...
        for i in range(1,self.k):
            # ...мы ищем номер ведущего элемента строки...
            ist = self.find_first(i)
            # ...и делаем xor для всех строк выше текущей
            for j in range(i-1,-1,-1):
                if(self.matr[j,ist]):
                    self.sum(i,j)
            # Таким образом зануляются все элементы над ведущим
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
    

    def delete_redundant_lines(self):
        '''Удаляет нулевые строки из матрицы.
        '''
        # Копия матрицы, с которой ведется работа
        temp = self.matr.copy()
        offset = 0 # сдвиг
        for i in range(self.k):
            # Если номер первой единицы в строке == кол-ву элементов самой строки (то есть единиц там нет)
            # Удаляем эту строку и сдвигаем указатель на 1
            if self.find_first(i) == self.n:
                temp = np.delete(temp, i - offset, 0)
                offset += 1
        # Перезаписываем матрицу на новую и очищенную
        self.matr = temp
        # Обновляем переменные-размеры матрицы
        self.n = temp.shape[1]
        self.k = temp.shape[0]

    
    def create_reduced_matr(self):
        '''
        Создает сокращенную матрицу X из текущей матрицы объекта
        :return: LinearCode(X) сокращенная матрица
        '''
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]
        return LinearCode(X)

                
    def create_verif_matr(self):
        '''Создает проверочную матрицу на основе порождающей.
        
        :return: LinearCode(H): проверочная матрица. 
        '''
        # Создаем новую матрицу в приведенном ступенчатом виде на основе текущей
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]

        # Единичная матрица
        Ind = np.identity(n = rrefed_matr.n-rrefed_matr.k, dtype = bool)

        # Создаем проверочную матрицу
        H = np.zeros(shape=(rrefed_matr.n,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        # сдвиги
        offset_i = 0
        offset_matr = 0
        # Заполняем ее значениями
        for i in range(H.shape[0]):
            # Если номер строки содержится в массиве с ведущими элементами, то заполняем строку из сокращенной матрицы
            # И прибавляем 1 к сдвигу для элементов единичной матрицы
            if i in leads:
                for j in range(H.shape[1]):
                    H[i,j] = X[i-offset_matr,j]
                offset_i += 1
            # Иначе заполняем строку из единичной матрицы
            # И прибавляем 1 к сдвику для матрицы сокращенной
            else:
                for j in range(H.shape[1]):
                    H[i,j] = Ind[i-offset_i,j]
                offset_matr += 1
        
        return LinearCode(H)
    
    def gen_codewords_summing(self):
        ''' Формирует набор слов путем сложения всех слов из порождающего множества

        :return: wordset: набор кодовых слов
        '''
        # Создаем пустой набор слов и доюавляем в него 0й элемент
        wordset = set()
        wordset.add(tuple((np.zeros(self.n, dtype=bool))))
        # Для i числа строк
        for i in range(1, self.k+1):
            # Массив длинны i всех возможных комбинаций строк 
            combinations = np.array(list(itertools.combinations(range(self.k), i)))
            for comb in combinations:
                # Для каждой комбинации
                # Создаем пустое слово
                word = np.zeros(self.n, dtype=bool)
                # Записываем в него сумму всех строк в текущей комбинации строк
                for j in comb:
                    word ^= self.matr[j]
                # Добавляем слово в набор слов
                wordset.add(tuple(word.tolist()))
        return wordset
    
    def gen_codewords_bin(self):
        ''' Формирует набор слов путем умножения всех двоичных слов длины k на G 

        :return: wordset: набор кодовых слов
        '''
        # Копия текущей матрицы
        G = LinearCode(self.matr)
        # Приводим в ступенчатому виду
        G.ref
        # Пустой набор слов
        wordset = set()
        # Набор всех возможных двоичных слов длины k
        binset = set(tuple(itertools.product((True,False),repeat=self.k)))
        # Для каждой комбинации
        for bin in binset:
            # Умножаем комбинацию на ступенчатую матрицу
            # И записываем слово в набор
            word = tuple((mult(bin,G.matr)).tolist())
            wordset.add(word)
        return wordset



### Часть 1
#### 4.1

In [159]:
def create_X_for_Golay():
    '''
    :return: Расширенный код Голея (24,12,8)
    '''
    B = [[1,1,0,1,1,1,0,0,0,1,0,1],
         [1,0,1,1,1,0,0,0,1,0,1,1],
         [0,1,1,1,0,0,0,1,0,1,1,1],
         [1,1,1,0,0,0,1,0,1,1,0,1],
         [1,1,0,0,0,1,0,1,1,0,1,1],
         [1,0,0,0,1,0,1,1,0,1,1,1],
         [0,0,0,1,0,1,1,0,1,1,1,1],
         [0,0,1,0,1,1,0,1,1,1,0,1],
         [0,1,0,1,1,0,1,1,1,0,0,1],
         [1,0,1,1,0,1,1,1,0,0,0,1],
         [0,1,1,0,1,1,1,0,0,0,1,1],
         [1,1,1,1,1,1,1,1,1,1,1,0]]
    return np.array(B,dtype=bool)

    
def get_Golay():
    '''
    Функция, создающая проверочную и порождающую матрицы для кода Голея (24,12,8)
    :return: G - порождающая матрица, H - проверочная матрица
    '''
    B = create_X_for_Golay() # Матрица B
    G = create_rref_matr(B) # Дополняем сокращенную матрицу единичной слева - получаем порождающую
    H = create_verif_matr(B) # Дополняем сокращенную матрицу единичной сверху - получаем проверочную
    return G, H

def check_Golay(w, H, G, B):
    '''
    Функция декодирования расширенного кода Голея
    :param w: передаваемое закодированное слово
    :param H: проверочная матрица
    :param G: порождающая матрица
    :param B: сокращенная матрица
    :return: u - декодированное слово
    '''
    s = mult(w, H) # Вычисляем синдром
    # Если вес синдрома <=3, дополняем синдром нулями справа и получаем исходное слово; возвращаем его 
    if np.sum(s) <= 3:
        u = np.hstack((s,np.zeros([B.shape[0]],dtype=bool)))
        return u
    
    # Если вес суммы синдрома и i-ой строки из B <= 2, дополняем эту сумму строкой с единственной единицей в позиции i справа
    # и получаем исходное слово; возвращаем его
    for i in range(B.shape[0]):
        if np.sum(s+B[i,:]) <= 2:
            e = np.zeros((B.shape[0]),dtype=bool)
            e[0, i] = True
            u = np.hstack((s+B[i,:],e),dtype=bool)
            return u
        
    sB = mult(s, B) # Вычисляем второй синдром
    # Если вес синдрома <= 3, дополняем его нулями слева и получаем исходное слово; возвращаем его
    if np.sum(sB) <= 3:
        u = np.hstack((np.zeros([B.shape[0]],dtype=bool),sB))
        return u
    
    # Если вес суммы синдрома и i-ой строки из B <= 2, дополняем эту сумму строкой с единственной единицей в позиции i слева
    # и получаем исходное слово; возвращаем его
    for i in range(B.shape[0]):
        if np.sum(sB+B[i,:]) <= 2:
            e = np.zeros((B.shape[0]),dtype=bool)
            e[0, i] = True
            u = np.hstack((e,sB+B[i,:]))
            return u
    # Ошибка не определена
    print("Unable to detect")
    return -1
    
    
print("G = ")
G,H = get_Golay()
print(LinearCode(G))
print("H = ")
print(LinearCode(H))

G = 
[1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 1]
[0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1]
[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]

H = 
[1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1]
[1 1 0 1 1 1 0 0 0 1 0 1]
[1 0 1 1 1 0 0 0 1 0 1 1]
[0 1 1 1 0 0 0 1 0 1 1 1]

#### 4.2

In [160]:
errors = np.eye(G.shape[1], dtype=bool)

sw = np.array([0,0,0,0,1,0,0,1,0,1,0,0],dtype=bool)  # Слово длины k 
w = mult(sw,G)  # Слово длины n
print("Получившееся слово: ", w)

word_err = w^errors[1] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

found_err = check_Golay(word_err,H,G,create_X_for_Golay()) # Нашли ошибку по таблице синдромов
print("U = ", found_err)
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == w)

# Обнаружил ошибку и исправил ее

Получившееся слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Слово с ошибкой:  [False  True False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
U =  [False  True False False False False False False False False False False
 False False False False False False False False False False False False]
Исправленное слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


In [161]:
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

found_err = check_Golay(word_err,H,G,create_X_for_Golay()) # Нашли ошибку по таблице синдромов
print("U = ", found_err)
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == w)

# Обнаружил ошибку и исправил ее

Получившееся слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Слово с ошибкой:  [False  True  True False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
U =  [False  True  True False False False False False False False False False
 False False False False False False False False False False False False]
Исправленное слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


In [162]:
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2]^errors[3] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

found_err = check_Golay(word_err,H,G,create_X_for_Golay()) # Нашли ошибку по таблице синдромов
print("U = ", found_err)
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == w)

# Обнаружил ошибку и исправил ее

Получившееся слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Слово с ошибкой:  [False  True  True  True  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
U =  [False  True  True  True False False False False False False False False
 False False False False False False False False False False False False]
Исправленное слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Совпадает ли с изначальным словом:  [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


In [163]:
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2]^errors[3]^errors[0] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

found_err = check_Golay(word_err,H,G,create_X_for_Golay()) # Нашли ошибку по таблице синдромов
print("U = ", found_err)
word_corr = word_err^found_err # Исправление ошибки
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == w)

# Не обнаружил ошибку

Получившееся слово:  [False False False False  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Слово с ошибкой:  [ True  True  True  True  True False False  True False  True False False
 False  True False  True  True  True  True  True False  True  True  True]
Unable to detect
U =  -1
Исправленное слово:  [-2 -2 -2 -2 -2 -1 -1 -2 -1 -2 -1 -1 -1 -2 -1 -2 -2 -2 -2 -2 -1 -2 -2 -2]
Совпадает ли с изначальным словом:  [False False False False False False False False False False False False
 False False False False False False False False False False False False]


### Часть 2
#### 4.3

In [ ]:
def inner_shape(matr, dimension):
    '''
    Проблема - у одномерных мессивов numpy shape возвращает один аргумент - его длину, а нужна возможность считывать 2 аргумента - длину (n) и высоту (1)
    Эта функция криво-косо решает проблему, позволяя считывать оба аргумента как для одномерных, так и для двумерных массивов
    :param matr: массив numpy
    :param dimension: измерение, длину в котором нужно считать
    :return: длина массива в измерении dimension
    '''
    ndims = matr.ndim
    if ndims == 1 and dimension == 1:
        return matr.shape[0]
    elif ndims == 1 and dimension == 0:
        return 1
    elif ndims == 2:
        return matr.shape[dimension]
    else:
        print("Unidefined dimension error")
        return -1
        
        

def create_G_for_RM(r,m):
    '''
    Рекурсивное построение порождающей матрицы для коди Рида-Миллера
    :param r: параметр r
    :param m: параметр m
    :return: G - порождающая матрица для параметров (r,m)
    '''
    # Для r = 0: G(0,m) - вектор единиц длины 2^m
    if(r==0):
        G = np.ones((pow(2,m)),dtype=bool)
        return G
    
    # Для r = m: G(m,m)
    if(r==m):
        # К матрице G(m-1,m) 
        G_up = create_G_for_RM(m-1,m)
        # Присоединяем снизу вектор нулей с единицей в конце; получаем G(m,m)
        G = np.vstack((G_up,np.zeros((inner_shape(G_up,1)),dtype=bool)))
        G[-1,-1] = True
        return G
    # Для 0 < r < m: 
    # G(r,m) = 
    # [ G(r,m-1)  G(r,m-1)   ]
    # [ 000...00  G(r-1,m-1) ]

    # np.shape => [n] k => error
    # inner_shape => [n] k => 1
    # np.shape => [n, k] => [n,1] [1,n]
    
    # Матрица G(r,m-1)
    G_rm1 = create_G_for_RM(r,m-1)
    # Матрица G(r-1,m-1)
    G_r1m1 = create_G_for_RM(r-1,m-1)
    # Матрица [ G(r,m-1)  G(r,m-1) ]
    G_up = np.hstack((G_rm1,G_rm1))
    # Матрица [ 000...00  G(r-1,m-1)]
    G_down = None
    if (inner_shape(G_r1m1,0)) == 1:
        # Если G(r-1,m-1) - одномерный вектор, то присоединяем к нему вектор нулей слева
        G_down = np.hstack((np.zeros((inner_shape(G_rm1,1)),dtype=bool),G_r1m1))
    else:
        # Если G(r-1,m-1) - двумерная матрица, то присоединяем к ней матрицу нулей слева
        G_down = np.hstack((np.zeros(((inner_shape(G_r1m1,0),inner_shape(G_rm1,1))),dtype=bool),G_r1m1))
    
    # Присоединяем к матрице [ G(r,m-1)  G(r,m-1) ] матрицу [ 000...00  G(r-1,m-1)]
    G = np.vstack((G_up,G_down))
    return G

In [165]:
print("G = ")
print(LinearCode(create_G_for_RM(1,3)))

G = 
[1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1]



In [166]:
def kronekers_mult(A,B):
    '''
    Произведение Кронекера
    :param A: матрица A
    :param B: матрица B
    :return: AxB
    '''
    
    # Составляем матрицу из i строк
    i_stack = None
    for i in range(A.shape[0]):
        # Составляем i-ую строчку матрицы AxB 
        j_stack = None
        for j in range(A.shape[1]):
            aijB = A[i,j]*B # Каждый элемент матрицы A умножается на матрицу B
            # Присоединятеся к концу i-ой строки
            if j_stack is None:
                j_stack = aijB
            else:
                j_stack = np.hstack((j_stack,aijB))
        # i-ая строка присоединяетсяя снизу к матрице AxB
        if i_stack is None:
            i_stack = j_stack
        else:
            i_stack = np.vstack((i_stack,j_stack))
    
    return i_stack

print(kronekers_mult(np.array([[1,1],[1,-1]]),np.array([[1,0],[0,1]])))           

[[ 1  0  1  0]
 [ 0  1  0  1]
 [ 1  0 -1  0]
 [ 0  1  0 -1]]


In [ ]:
def create_Him_for_RM(i,m):
    '''
    Функция, составляющая матрицу H^i_m = I_(2^(m-1)) x H x I_(2^(i-1))
    :param i: параметр i
    :param m: параметр m
    :return: матрица H^i_m
    '''
    # H = 
    # [1  1]
    # [1 -1]
    Imi = np.eye(pow(2,m-i),dtype=int) # Матрица I_(2^(m-1))
    Ii1 = np.eye(pow(2,i-1),dtype=int) # Матрица I_(2^(i-1))
    H = np.array([[1,1],[1,-1]],dtype=int) # Матрица H
    Him = kronekers_mult(kronekers_mult(Imi,H),Ii1) # Получаем H^i_m путем перемножения этих трех матриц
    return Him

print("Him = ")
print(create_Him_for_RM(3,3))

Him = 
[[ 1  0  0  0  1  0  0  0]
 [ 0  1  0  0  0  1  0  0]
 [ 0  0  1  0  0  0  1  0]
 [ 0  0  0  1  0  0  0  1]
 [ 1  0  0  0 -1  0  0  0]
 [ 0  1  0  0  0 -1  0  0]
 [ 0  0  1  0  0  0 -1  0]
 [ 0  0  0  1  0  0  0 -1]]


In [168]:
G, Him = create_G_for_RM(1,3), create_Him_for_RM(1,3)

print("Him = ")
print(Him)
print("G = ")
print(LinearCode(G))


Him = 
[[ 1  1  0  0  0  0  0  0]
 [ 1 -1  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0]
 [ 0  0  0  0  1  1  0  0]
 [ 0  0  0  0  1 -1  0  0]
 [ 0  0  0  0  0  0  1  1]
 [ 0  0  0  0  0  0  1 -1]]
G = 
[1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1]



#### 4.4

In [ ]:
def to_binary_np(n,m):
    '''
    Функция, возвращающая двоичное представление числа с младшими битами в начале
    :param n: число
    :param m: длина числа в битах
    :return: обратное двоичное представление числа
    '''
    # Такс, справа налево:
    # np.binary_repr возвращает двоичное представление числа длины m в виде строки
    # Строка преобразуется в список
    # Этот список преобразовывается в обратный, с младшими битами слева
    # После чего список преобразуется в массив numpy со значениями типа int
    return np.array(list(reversed(list(np.binary_repr(n, width=m)))), dtype=int)


def check_RM(w,m):
    '''
    Функция, выполняющая декодирование для RM(1,m)
    :param w: закодированное слово
    :param m: параметр m
    :return: декодированное слово
    '''
    # w_new формируется заменой всех 0 на -1
    w_new = np.zeros(w.shape[0],dtype=int)
    for i in range(w.shape[0]):
        w_new[i] = 1 if w[i] else -1 
    
    # Вычисляется w_1 = w_new*H^1_m
    wi = w_new @ create_Him_for_RM(1,m)
    # Для всех i: w_i = w_(i-1)*H^i_m
    for i in range(2,m+1):
        wi = wi @ create_Him_for_RM(i,m)
    # Находим позицию j наибольшего по значению элемента
    j = (np.abs(wi)).argmax()
    
    # Если этот элемент положительный, то его обратное двоичное представление дополняется единицей слева
    if(wi[j]>=0):
        return np.hstack((1,to_binary_np(j,m)))
    else:
        # Если этот элемент отрицательный, то его обратное двоичное представление дополняется нулем слева
        return np.hstack((0,to_binary_np(j,m)))
    # Полученное слово и будет исходным сообщением
    

In [170]:
sw = [False,False,True,True] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_corr = check_RM(w,3) # Декодированное слово
print("Декодированное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным

Получившееся слово:  [False False  True  True  True  True False False]
Декодированное слово:  [0 0 1 1]
Совпадает ли с изначальным словом:  [ True  True  True  True]


In [171]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,3) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным, ошибка исправлена

Получившееся слово:  [False False  True  True  True  True False False]
Слово с ошибкой:  [False  True  True  True  True  True False False]
Исправленное слово:  [0 0 1 1]
Совпадает ли с изначальным словом:  [ True  True  True  True]


In [172]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True] # Исходное слово  
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,3) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово не совпадает с исходным, ошибка не исправлена

Получившееся слово:  [False False  True  True  True  True False False]
Слово с ошибкой:  [False  True False  True  True  True False False]
Исправленное слово:  [0 1 0 0]
Совпадает ли с изначальным словом:  [ True False False False]


#### 4.5

In [173]:
m = 4

G, Him = create_G_for_RM(1,m), create_Him_for_RM(1,m)

print("Him = ")
print(Him)
print("G = ")
print(LinearCode(G))


Him = 
[[ 1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1 -1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1 -1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -1]]
G = 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]



In [174]:
sw = [False,False,True,True,False] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_corr = check_RM(w,m) # Декодированное слово
print("Декодированное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным

Получившееся слово:  [False False  True  True  True  True False False False False  True  True
  True  True False False]
Декодированное слово:  [0 0 1 1 0]
Совпадает ли с изначальным словом:  [ True  True  True  True  True]


In [175]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True,False] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,m) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным, ошибка исправлена

Получившееся слово:  [False False  True  True  True  True False False False False  True  True
  True  True False False]
Слово с ошибкой:  [False  True  True  True  True  True False False False False  True  True
  True  True False False]
Исправленное слово:  [0 0 1 1 0]
Совпадает ли с изначальным словом:  [ True  True  True  True  True]


In [176]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True,False] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,m) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным, ошибка исправлена

Получившееся слово:  [False False  True  True  True  True False False False False  True  True
  True  True False False]
Слово с ошибкой:  [False  True False  True  True  True False False False False  True  True
  True  True False False]
Исправленное слово:  [0 0 1 1 0]
Совпадает ли с изначальным словом:  [ True  True  True  True  True]


In [177]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True,False] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2]^errors[3] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,m) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово совпадает с исходным, ошибка исправлена

Получившееся слово:  [False False  True  True  True  True False False False False  True  True
  True  True False False]
Слово с ошибкой:  [False  True False False  True  True False False False False  True  True
  True  True False False]
Исправленное слово:  [0 0 1 1 0]
Совпадает ли с изначальным словом:  [ True  True  True  True  True]


In [178]:
errors = np.eye(G.shape[1], dtype=bool)

sw = [False,False,True,True,False] # Исходное слово
w = mult(sw,G) # Закодированное слово
print("Получившееся слово: ", w)

word_err = w^errors[1]^errors[2]^errors[3]^errors[0] # Внесение ошибки
print("Слово с ошибкой: ", word_err)

word_corr = check_RM(word_err,m) # Декодированное слово
print("Исправленное слово: ", word_corr)
print("Совпадает ли с изначальным словом: ", word_corr == sw)
# Декодированное слово не совпадает с исходным, ошибка не исправлена

Получившееся слово:  [False False  True  True  True  True False False False False  True  True
  True  True False False]
Слово с ошибкой:  [ True  True False False  True  True False False False False  True  True
  True  True False False]
Исправленное слово:  [1 0 1 0 0]
Совпадает ли с изначальным словом:  [False  True  True False  True]
